# Aspects

- Adds global behavior to the application
- Re-usable block of code that can be injected into the application during runtime
- Leverages AspectJ
- Byte code modification (Runtime interweaving)
- Dynamic proxy-based
- Common applications
    - logging and tracing
    - Transaction management
    - Caching
    - Security
- Parts of Spring Aspect
    - Join Point : 
        - Represents these specific points in the execution of a program where advice can be applied.
        - Business logic where aspect can be applied 
        - You apply the aspect here
    - Pointcut : 
        - Specifies certain conditions or criteria that define when certain pieces of code (advice) should be executed.
        - Select the joinpoint for the cross-cutting concern 
        - This is the trigger of aspect 
        - signature: `designator("returntype packageName.className.methodName(argument)" )`
        - common designators : 
            - `execution` (expression for matching method execution), 
            - `within` (expression for matching within certain type), 
            - `target` (expression for matching specific type), 
            - `@annotation` (expression for matching specific annotation)
    - Advice : 
        - Represents the actual code or instructions that need to be executed when the conditions specified by the pointcut are met.
        - The code that is applied to the join point when it is selected by the pointcut (cross-cutting concern )
        - This is the aspected behavior itself


# Logging using AOP 

### `pom.xml` dependency

```
<dependency>
    <groupId>org.aspectj</groupId>
    <artifactId>aspectjweaver</artifactId>
    <version>1.9.5</version>
</dependency>
<dependency>
    <groupId>org.slf4j</groupId>
    <artifactId>slf4j-log4j12</artifactId>
    <version>1.7.30</version>
</dependency>
```

### `log4j.properties` file

```
log4j.rootLogger=INFO, stdout

log4j.appender.stdout=org.apache.log4j.ConsoleAppender
log4j.appender.stdout.Target=System.out
log4j.appender.stdout.layout=org.apache.log4j.PatternLayout
log4j.appender.stdout.layout.ConversionPattern=%d{yyyy-MM-dd HH:mm:ss} %-5p %c{1} - %m%n
```

### Enable AOP in `ApplicationConfig.java`

```
@Configuration
@PropertySource("classpath:application.properties")
@ComponentScan(basePackages = "com.frankmoley.lil.fid")
@EnableAspectJAutoProxy // Enable AOP
public class ApplicationConfig {

}
```

### Create annotation `Loggable` (This will act as a pointcut)

```
@Target(ElementType.METHOD) // annotation should work on methods
@Retention(RetentionPolicy.RUNTIME) // annotation should be loaded during runtime
public @interface Loggable {
}

```

### Create service `GreetingService.java` (This is where Joinpoint resides)

```
@Service
public class GreetingService {

    @Value("${app.greeting}")
    private String greeting;

    public GreetingService(){
        super();
    }

    @Loggable // This is the joinpoint : Where the aspected behavior is stitched
    public String getGreeting(String name){
        return greeting + " " + name;
    }
}

```

### Create Aspect class `LoggingAspect.java` (This will act as advise )

```


@Component
@Aspect // ensure for component scanning
public class LoggingAspect {

    private static final Logger LOGGER = LoggerFactory.getLogger(LoggingAspect.class);

    @Pointcut("@annotation(Loggable)") // connect with the annotation. The annotation is the pointcut (will trigger this aspect)
    public void executeLogging(){} // The original Advise : the logic that will run when annotation is called

    @Before(value = "executeLogging()")  // Pre-Advise: the logic that will run before execution of original advise
    public void logMethodCall(JoinPoint joinPoint){
        StringBuilder message = new StringBuilder("Method: ");
        message.append(joinPoint.getSignature().getName());
        Object[] args = joinPoint.getArgs();
        if (null!=args && args.length>0){
            message.append(" args=[ | ");
            Arrays.asList(args).forEach(arg->{
                message.append(arg).append(" | ");
            });
            message.append("]");
        }
        LOGGER.info(message.toString());
    }
    
    @AfterReturning(value = "executeLogging()", returning = "returnValue") // Post-Advise: the logic that will run after execution of original advise
    public void logMethodCall(JoinPoint joinPoint, Object returnValue){
        StringBuilder message = new StringBuilder("Method: ");
        message.append(joinPoint.getSignature().getName());
        Object[] args = joinPoint.getArgs();
        if (null!=args && args.length>0){
            message.append(" args=[ | ");
            Arrays.asList(args).forEach(arg->{
                message.append(arg).append(" | ");
            });
            message.append("]");
        }
        if(returnValue instanceof Collection){
            message.append(", returning: ").append(((Collection)returnValue).size()).append(" instance(s)");
        }else{
            message.append(", returning: ").append(returnValue.toString());
        }

        LOGGER.info(message.toString());
    }
    

    @Around(value = "executeLogging()") // Most Flexible : Anything before original advise, anything after original advise returning value, anything after original advise returning exception
    public Object logMethodCall(ProceedingJoinPoint joinPoint) throws Throwable {
        long startTime = System.currentTimeMillis();
        Object returnValue = joinPoint.proceed();
        long totalTime = System.currentTimeMillis()-startTime;
        StringBuilder message = new StringBuilder("Method: ");
        message.append(joinPoint.getSignature().getName());
        message.append(" totalTime: ").append(totalTime).append("ms");
        Object[] args = joinPoint.getArgs();
        if (null!=args && args.length>0){
            message.append(" args=[ | ");
            Arrays.asList(args).forEach(arg->{
                message.append(arg).append(" | ");
            });
            message.append("]");
        }
        if(returnValue instanceof Collection){
            message.append(", returning: ").append(((Collection)returnValue).size()).append(" instance(s)");
        }else{
            message.append(", returning: ").append(returnValue.toString());
        }

        LOGGER.info(message.toString());
        return returnValue;
    }


}

```

# Example 2 : Simple AOP 

```
// pom.xml dependencies for AspectJ and Logging
<dependencies>
    <!-- Spring dependencies -->
    <dependency>
        <groupId>org.springframework.boot</groupId>
        <artifactId>spring-boot-starter</artifactId>
    </dependency>
    <dependency>
        <groupId>org.springframework.boot</groupId>
        <artifactId>spring-boot-starter-aop</artifactId>
    </dependency>

    <!-- Logging dependencies -->
    <dependency>
        <groupId>org.slf4j</groupId>
        <artifactId>slf4j-api</artifactId>
    </dependency>
    <dependency>
        <groupId>ch.qos.logback</groupId>
        <artifactId>logback-classic</artifactId>
    </dependency>
</dependencies>

// ApplicationConfig.java for ApplicationContext configuration
@Configuration
@ComponentScan(basePackages = "com.example.myapp")
@EnableAspectJAutoProxy
public class ApplicationConfig {
}

// Custom annotation to use as PointCut trigger
@Retention(RetentionPolicy.RUNTIME)
@Target(ElementType.METHOD)
public @interface LogThis {
}

// Service class as Joinpoint : Where the Aspected behavior is applied through annotation
@Service
public class MyService {

    @LogThis
    public void doSomething() {
        System.out.println("Doing something...");
    }
}

// The aspect class advises : The custom behavior that is applied to the JoinPoints through annotation calls
@Aspect
@Component
public class LoggingAspect {

    @Pointcut("@annotation(com.example.myapp.annotation.LogThis)") // Setting up the annotation as pointcut
    private void logThisMethods() {} // Root advise method

    @Before("logThisMethods()") // this method will be called before root advise method
    public void logBefore(JoinPoint joinPoint) {
        System.out.println("Before executing " + joinPoint.getSignature().toShortString());
    }

    @After("logThisMethods()") // this method will be called after root advise method
    public void logAfter(JoinPoint joinPoint) {
        System.out.println("After executing " + joinPoint.getSignature().toShortString());
    }

    @Around("logThisMethods()") // FLEXIBLE: take care before, after and during exception while processing root advise method 
    public Object logAround(ProceedingJoinPoint joinPoint) throws Throwable {
        System.out.println("Before executing " + joinPoint.getSignature().toShortString());
        Object result = joinPoint.proceed();
        System.out.println("After executing " + joinPoint.getSignature().toShortString());
        return result;
    }

    @AfterReturning(pointcut = "logThisMethods()", returning = "result")  // this method will be called after root advise method
    public void logAfterReturning(JoinPoint joinPoint, Object result) {
        System.out.println("After returning from " + joinPoint.getSignature().toShortString() + ", result: " + result);
    }

    @AfterThrowing(pointcut = "logThisMethods()", throwing = "exception") // this method will take care any method exception
    public void logAfterThrowing(JoinPoint joinPoint, Throwable exception) {
        System.out.println("After throwing exception from " + joinPoint.getSignature().toShortString() + ", exception: " + exception.getMessage());
    }
}
```